## Adaptive Resonance Theory
Feedback mechanism + Competitive-learning scheme
#### ART1

In [2]:
import keras
from keras.datasets import mnist
import numpy as np

Using Theano backend.


#### 讀取 MNIST
see https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py

In [10]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
_, img_rows, img_cols = x_train.shape
print ('原始 shape = (資料數目, #row, #col)')
print ('#train = ', x_train.shape[0], ', #test = ', x_test.shape[0])
print ('image rows = ', img_rows, ', image_cols = ', img_cols)
# flatten
x_train = np.reshape(x_train, (x_train.shape[0], img_rows*img_cols))
x_test = np.reshape(x_test, (x_test.shape[0], img_rows*img_cols))
print ('image map 轉 vector')
print ('new x_train.shape = ', x_train.shape)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

原始 shape = (資料數目, #row, #col)
#train =  60000 , #test =  10000
image rows =  28 , image_cols =  28
image map 轉 vector
new x_train.shape =  (60000, 784)


因為ART1只適用於"binary patterns"，將圖檔轉為binary image
$\left[0,255\right] \rightarrow \left[0,1\right]$

In [11]:
def Convert2Binary(x, thre = 128):
    idx = x[:,:] < thre
    x[idx] = 0
    idx = x[:,:] > 0
    x[idx] = 1
    return x

x_train = Convert2Binary(x_train)
x_test = Convert2Binary(x_test)

#### A. Preparation
1. Determin $F_1$ and $F_2$ nodes: $dim(F_1, F_2) = (M,N)$
2. Determin parameter values: $A_1\leq 0,~C_1\leq 0,~P_1\leq 0,~L>1$
  * $A$: Orienting subsystem, (1 reset node)

In [ ]:
class ART1(object):
    def __init__(self, M = 5, N = 6, L = 3, rho = 0.9, A = 1, B = 1.5, C = 5, D=0.9):
        self.A = 1;